In [ ]:
import os
import pickle
from src.tools import plot_hyperparam_results

BASE_DIR = "/content/drive/MyDrive/tomato"
CKPT_DIR = os.path.join(BASE_DIR, "checkpoints")

with open(os.path.join(CKPT_DIR, "hyperparam_results.pkl"), "rb") as f:
    results = pickle.load(f)

# 绘制验证集准确率对比
plot_hyperparam_results(results, metric="val_acc")

# 绘制训练集损失对比
plot_hyperparam_results(results, metric="train_loss")
